In [2]:
import requests
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import img_to_array, get_file
from tensorflow.keras.preprocessing.image import load_img
from skimage.transform import resize

### Load Labels

In [9]:
LABELS_PATH = './labels.txt'

with open(LABELS_PATH, 'rt') as f:
    labels = [label[:-1] for label in f.readlines()]

print('Num of Labels: %s' % len(labels))
print('Labels Sample: ', labels[:5])

Num of Labels: 1000
Labels Sample:  ['tench, Tinca tinca', 'goldfish, Carassius auratus', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'tiger shark, Galeocerdo cuvieri', 'hammerhead, hammerhead shark']


## Loading and preprocessing

In [11]:
INPUT_SHAPE = (224, 224, 3)
def load_and_preprocess(image_path):
    img = load_img(image_path)
    img = img_to_array(img)
    img = resize(img, INPUT_SHAPE)
    img = preprocess_input(img)
    return img

## Load the model 

In [14]:
vgg = VGG16(input_shape=INPUT_SHAPE, weights='imagenet', include_top=True)

tf.keras.utils.plot_model(vgg, show_shapes=True)

107642880/553467096 [====>.........................] - ETA: 7:33

In [ ]:
SELECT_LAYERS = [1, 3, 5, 7, 8, 10]

def get_layers_output(img):
  input_layer = vgg.layers[0]
  x = input_layer(np.expand_dims(img, axis=0))
  outputs = [x[0]]
  for layer in vgg.layers[1:]:
    x = layer(x)
    outputs.append(x[0])
  
  outputs = np.array(outputs)
  return outputs[SELECT_LAYERS]

def plot_layer_output(feature_maps, layer_idx, first=3):
  fig = plt.figure(figsize=(first * 3, 4))
  fig.suptitle('Layer %s' % layer_idx)
  for idx in range(first):
    sub_img = feature_maps[:, : , idx]
    ax = plt.subplot(1, first, idx + 1)
    ax.imshow(sub_img) 
    ax.set_title(feature_maps.shape)
    ax.axis('off')
  plt.show()

def plot_feature_maps(layers_output):
  for idx, output in zip(SELECT_LAYERS, layers_output):
    plot_layer_output(output, idx, first=10)